In [53]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

In [34]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = 'langsmith_api_key'

In [35]:
from langchain_community.document_loaders import UnstructuredPDFLoader

In [36]:
local_path = "C:/Documents/AI/GenAI/PDFs/Claims_Flow.pdf"

# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

In [37]:
len(data)

1

In [38]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [39]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(data)

In [40]:
len(chunks)

52

In [41]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings:   0%|          | 0/52 [00:00<?, ?it/s]

OllamaEmbeddings: 100%|██████████| 52/52 [02:33<00:00,  2.96s/it]


In [42]:
# Load environment variable from .env file
load_dotenv()

True

In [43]:
# access environment variable
openai_api_key = os.getenv("OPENAI_API_KEY")

In [44]:
# create an llm object
openai_llm = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-3.5-turbo", temperature=0.5)

In [45]:
# # LLM from Ollama
# local_model = "mistral"
# ollama_llm = ChatOllama(model=local_model)

In [46]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}
    """,
)

In [47]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    openai_llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context in detail.
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [48]:
# ollama_chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | prompt
#     | ollama_llm
#     | StrOutputParser()
# )

In [49]:
openai_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | openai_llm
    | StrOutputParser()
)

In [50]:
# ollama_chain.invoke("What are sanity checks performed in OHI? Explain in 200 words.")

In [51]:
openai_chain.invoke("What is this document about?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.33s/it]


'This document is a Software Requirements Specification (SRS) document that provides a detailed overview of a software product, its parameters, goals, target audience, user interface, hardware and software requirements. It also defines how the client, team, and audience see the product and its functionality. The document assists designers and developers in the software delivery lifecycle (SDLC) processes.'